In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np, os
from scripts.core import *
from fastai.learner import Learner
import fastai
from torch.utils.data import Dataset
from scripts.models import *
from scripts.learners import modelLearner, ParallelLearner

In [3]:
class Vid100Datasat(Dataset):
    def __init__(self, train=True):
        super().__init__()
        import random
        self.train=train
        PTH = '/home/sufiyan/Common_data/mtp2/makeDataset/Sub Video/data/Mercedes_224_224_Manual_annonate_8sec_LSTM_short_005_'
        PTH_L=os.path.join(PTH,"LEFT")
        PTH_R=os.path.join(PTH,"RIGHT")
        PTH_C=os.path.join(PTH,"CLEAR")
        left_vids =[os.path.join(d,f) for d,_,fs in os.walk(PTH_L) for f in fs if f.split(".")[-1]=='npy']
        right_vids=[os.path.join(d,f) for d,_,fs in os.walk(PTH_R) for f in fs if f.split(".")[-1]=='npy']
        clear_vids=[os.path.join(d,f) for d,_,fs in os.walk(PTH_C) for f in fs if f.split(".")[-1]=='npy']
        self.fileNames=left_vids+right_vids+clear_vids
        np.random.shuffle(self.fileNames)
     
    @staticmethod
    def stoi(label):
        stoi={"CLEAR":0, "LEFT":1, "RIGHT":2}
        return stoi[label]
    
    def __getitem__(self,i):
        X=np.load(self.fileNames[i])
        y=self.get_label(i)
        y=np.asarray([self.stoi(label) for label in y])
        X=np.expand_dims(X,1)
        X=np.concatenate((X,X,X), axis=1)
        return (torch.from_numpy(X), y)
    def __len__(self): return len(self.fileNames)
    
    def get_label(self, i):
        filename=self.fileNames[i]
        if self.train:
              return np.load(filename.replace('/data/','/label/'))
        else: return np.load(filename.replace('/data/','/val_label/'))

In [4]:
trainLoader=torch.utils.data.DataLoader(Vid100Datasat(train=True) ,batch_size=1, shuffle=True)
validLoader=torch.utils.data.DataLoader(Vid100Datasat(train=False),batch_size=1, shuffle=True)

In [5]:
learner=ParallelLearner([modelLearner(C3D_resnet_ConvLSTM2D(3, 3), loss_fn=nn.MSELoss, optim=torch.optim.SGD, modelName="MSE_22Oct", lr=0.0001, is_depth=True)], epochs=2, trainLoader=trainLoader, validLoader=validLoader, printEvery=1)

In [6]:
learner.train()

100%|██████████| 793/793 [20:17<00:00,  1.54s/it]



**************************************************
Epoch: 0   Time Elapsed: 1217.7005751132965
lr: 0.0001      trainLoss: 219.60212153196335


  0%|          | 0/793 [00:00<?, ?it/s]

lr: 0.0001      MSE testLoss: 192.5027190297842


100%|██████████| 793/793 [20:23<00:00,  1.54s/it]



**************************************************
Epoch: 1   Time Elapsed: 3107.680511236191
lr: 0.0001      trainLoss: 198.4656713604927
lr: 0.0001      MSE testLoss: 176.2449575960636


In [15]:
c=learner.learners[0].confusion_matrix

#### The Model has overfitted to Clear Label

In [17]:
c.value()

array([[269718,      0,      0],
       [ 23750,      0,      0],
       [ 23732,      0,      0]], dtype=int32)